# Schema Alignment

# Datasets Analysis

In [ ]:
!pip install pandas -q
!pip install numpy -q
!pip install pandas-profiling -q
!pip install ipywidgets -q

In [ ]:
import os
import pandas as pd
import numpy as np
import time

In [ ]:
t_start = time.time()

## wissel-ariregister

In [ ]:
df_aziende = pd.read_csv("./src/datasets/wissel-ariregister.rik.ee/wissel-aziende-ariregister.rik.ee.csv")
df_activity = pd.read_csv("./src/datasets/wissel-ariregister.rik.ee/wissel-activity-ariregister.rik.ee.csv")
df_partners = pd.read_csv("./src/datasets/wissel-ariregister.rik.ee/wissel-partners-ariregister.rik.ee.csv")
df_rappresentanti = pd.read_csv("./src/datasets/wissel-ariregister.rik.ee/wissel-rappresentanti-ariregister.rik.ee.csv")

In [ ]:
df_aziende.rename(columns={'ID':'ID azienda'}, inplace=True)
toDrop = ["URL", "Code", "Legal form", "Status", "Registration Date", "Deletion Date"]
df_aziende = df_aziende.drop(toDrop, axis=1)
df_aziende.head()

In [ ]:
toDrop = ["EMTAK Code", "NACE Code", "Source"]
df_activity = df_activity.drop(toDrop, axis=1)
df_activity.head()

In [ ]:
df_partners.head()

In [ ]:
df_rappresentanti.head()

In [ ]:
df3 = pd.merge(df_aziende, df_activity, how='left', on='ID azienda')

In [ ]:
df3.head()

In [ ]:
df3.to_csv("./src/datasets/wissel-ariregister.rik.ee.csv")

## wissel-companiesmarketcap

In [ ]:
df_companies_aziende = pd.read_csv("./src/datasets/wissel-companiesmarketcap.com/wissel-aziende-companiesmarketcap.com.csv")
df_companies_categorie = pd.read_csv("./src/datasets/wissel-companiesmarketcap.com/wissel-categorie-companiesmarketcap.com.csv")

In [ ]:
df_companies_aziende.head()

In [ ]:
df_companies_aziende.rename(columns={'Unnamed: 0':'ID azienda'}, inplace=True)
toDrop = ["ID", "URL"]
df_companies_aziende = df_companies_aziende.drop(toDrop, axis=1)
df_companies_aziende.head()

In [ ]:
for i in range(0, 5160):
    df_companies_aziende.loc[i, "ID azienda"] += 1401

In [ ]:
for i in range(5160, 6560):
    df_companies_aziende.loc[i, "ID azienda"] += 1

In [ ]:
df_companies_aziende.sort_values(["ID azienda"], 
                    axis=0,
                    ascending=[True], 
                    inplace=True)

In [ ]:
df_companies_aziende.head()

In [ ]:
df_companies_categorie.rename(columns={'Company ID':'ID azienda'}, inplace=True)
df_companies_categorie.head()

In [ ]:
df_companies_aziende['ID azienda'] = pd.to_numeric(df_companies_aziende['ID azienda'], errors='coerce').fillna(0).astype(int)

In [ ]:
df_companies_aziende.head()

In [ ]:
df4 = pd.merge(df_companies_aziende, df_companies_categorie, how='left', on='ID azienda')
df4.head()

In [ ]:
df4 = df4.groupby(["ID azienda","Name","Company code", "Marketcap", "Share price", "Earnings", "Revenue", "Shares", "Employees"])['Category'].agg(list).reset_index()

In [ ]:
df4.head()

In [ ]:
df4.to_csv("./src/datasets/wissel-companiesmarketcap.com.csv")

## FR_sole24ore.csv

In [ ]:
fr_sole24 = pd.read_csv("./src/datasets/FR_sole24ore.csv")
if fr_sole24.columns[len(fr_sole24.columns)-1] == "name":
    fr_sole24 = fr_sole24.drop(fr_sole24.columns[0], axis=1)
else:
    fr_sole24 = pd.read_csv("./src/datasets/FR_sole24ore.csv", header=None)

In [ ]:
fr_sole24.columns = ["address", "city", "sector", "employees", "name"]
fr_sole24.head()

In [ ]:
fr_sole24.to_csv("./src/datasets/FR_sole24ore.csv")

## Files

In [ ]:
source = 'src/datasets/'
files_list = os.listdir(source)
files_list.sort()
files_list = [source + x for x in files_list if os.path.isfile(source+x)]

In [ ]:
# print(files_list)
final_files = []
len(files_list)

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

file_ext = {}
columns_file = {}
files_columns = {}
for i in files_list:
    #print("File name: " + i)
    #print(i[len(i)-3:])
    if i[len(i)-3:] == "csv":
        if "csv" not in file_ext:
            file_ext["csv"] = 0
        file_ext["csv"] += 1

        current_file = pd.read_csv(i, encoding="utf-8", encoding_errors="ignore")
        # print(file_csv.columns)


    elif i[len(i)-4:] == "json":
        if "json" not in file_ext:
            file_ext["json"] = 0
        file_ext["json"] += 1

        current_file = pd.read_json(i)
        # print(file_json.head())
        
    elif i[len(i)-5:] == "jsonl":
        if "jsonl" not in file_ext:
            file_ext["jsonl"] = 0
        file_ext["jsonl"] += 1

        current_file = pd.read_json(i, lines=True)
        # print(file_jsonl.head())
        
    elif i[len(i)-3:] == "xls":
        if "xls" not in file_ext:
            file_ext["xls"] = 0
        file_ext["xls"] += 1

        current_file = pd.read_excel(i)
        # print(file_xls.head())
        
    elif i[len(i)-4:] == "xlsx":
        if "xlsx" not in file_ext:
            file_ext["xlsx"] = 0
        file_ext["xlsx"] += 1
        
        # Genera warning
        current_file = pd.read_excel(i)
        # print(file_xlsx.head())
        
    else:
        print("file error: " + i)

    if i not in files_columns:
        files_columns[i] = current_file.columns


    for c in current_file.columns:
        if c.lower() not in columns_file:
            columns_file[c.lower()] = 0
        columns_file[c.lower()] += 1

In [ ]:
file_ext

In [ ]:
files_columns

## Check columns

In [ ]:
import json
# with open('columns_to_remove.json', 'w', encoding='utf8') as f:
#     json.dump(columns_to_remove, f, ensure_ascii=False)

# with open('columns_to_rename.json', 'w', encoding='utf8') as f:
#     json.dump(columns_to_rename, f, ensure_ascii=False)

In [ ]:
def columns_to_remove_not_remove(columns):
    with open('columns_to_remove.json', 'r', encoding='utf8') as f:
        columns_to_remove = json.load(f)

    r = []
    l = []
    print("Columns to remove:")
    for c in columns:
        if c.lower() in columns_to_remove:
            print(c)
            r.append(c)
        else:
            l.append(c)
    return (r, l)

In [ ]:
def rename_columns(columns):
    with open('columns_to_rename.json', 'r', encoding='utf8') as f:
        columns_to_rename = json.load(f)
    
    l = []
    print("Columns to rename:")
    for c in columns:
        error=True
        for name_column, list_strings in columns_to_rename.items():
            if c.lower() == name_column:
                l.append(name_column)
                error=False
            elif c.lower() in list_strings:
                print(c + " -> " + name_column)
                l.append(name_column)
                error=False    
        
        if error: 
            l.append(c.lower())
            print("Column error: " + c)
    return l

In [ ]:
cleaned_files = []

## Convert market cap and revenue

In [ ]:
def convert_market_cap_revenue(val):
    if "T" in val or "Trillion" in val:
        val = val.replace('$','').replace("Trillion", '').replace('USD', '').replace('US', '').replace('T','').strip()
        return int(float(val)*(10**12))
    elif "B" in val or "Billion" in val:
        val = val.replace('$','').replace("Billion", '').replace('USD', '').replace('US', '').replace('B','').strip()
        return int(float(val)*(10**9))
    elif "M" in val:
        val = val.replace('$','').replace("Million", '').replace('USD', '').replace('US', '').replace('M','').strip()
        return int(float(val)*(10**6))
    else:
        return np.nan

## DDD

### DDD-cbinsight.com.csv

In [ ]:
files_list[0]

In [ ]:
ddd_cbinsight = pd.read_csv(files_list[0])
ddd_cbinsight.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(ddd_cbinsight)

In [ ]:
ddd_cbinsight = ddd_cbinsight.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_cbinsight.columns)
ddd_cbinsight.columns = to_rename

In [ ]:
ddd_cbinsight = ddd_cbinsight.replace('None', np.nan)
ddd_cbinsight.founded = ddd_cbinsight.founded.astype(object).astype('Int64')

ddd_cbinsight.head()

In [ ]:
cleaned_files.append(ddd_cbinsight)

### DDD-companiesmarketcap.com.csv

In [ ]:
files_list[1]

In [ ]:
ddd_companies = pd.read_csv(files_list[1])
ddd_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(ddd_companies)

In [ ]:
ddd_companies = ddd_companies.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_companies.columns)
ddd_companies.columns = to_rename

In [ ]:
ddd_companies.marketcap = ddd_companies.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
ddd_companies = ddd_companies.replace('None', np.nan)
ddd_companies.marketcap = ddd_companies.marketcap.astype(object).astype('Int64')

ddd_companies.head()

In [ ]:
cleaned_files.append(ddd_companies)

### DDD-ft.com.csv

In [ ]:
files_list[2]

In [ ]:
ddd_ft = pd.read_csv(files_list[2])
ddd_ft.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(ddd_ft)

In [ ]:
ddd_ft = ddd_ft.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_ft.columns)
ddd_ft.columns = to_rename

In [ ]:
ddd_ft = ddd_ft.replace('None', np.nan)
ddd_ft.revenue = ddd_ft.revenue.apply(lambda x : x.replace(',',''))

ddd_ft.revenue = ddd_ft.revenue.astype(object).astype('Int64')
ddd_ft.employees = ddd_ft.employees.astype(object).astype('Int64')
ddd_ft.founded = ddd_ft.founded.astype(object).astype('Int64')
ddd_ft.head()

In [ ]:
cleaned_files.append(ddd_ft)

### DDD-teamblind.com.csv

In [ ]:
files_list[3]

In [ ]:
ddd_teamblind = pd.read_csv(files_list[3])
ddd_teamblind.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(ddd_teamblind)

In [ ]:
ddd_teamblind = ddd_teamblind.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_teamblind.columns)
ddd_teamblind.columns = to_rename

In [ ]:
ddd_teamblind.country = ddd_teamblind.country.apply(lambda x: x.split(',', 1)[0])
ddd_teamblind.employees = ddd_teamblind.employees.apply(lambda x: x.replace(',', '').replace('to ', '').replace(' employees', '').replace('+', '').split(' ', 1)) 
ddd_teamblind.employees = ddd_teamblind.employees.apply(lambda x: int(sum([int(i) for i in x])/2) if x[0] != 'None' else np.nan)
ddd_teamblind = ddd_teamblind.replace('None', np.nan)
ddd_teamblind.employees = ddd_teamblind.employees.astype(object).astype('Int64')
ddd_teamblind.founded = ddd_teamblind.founded.astype(object).astype('Int64')
ddd_teamblind.head()

In [ ]:
cleaned_files.append(ddd_teamblind)

## FR

### FR_campaignindia.csv

In [ ]:
files_list[4]

In [ ]:
fr_india = pd.read_csv(files_list[4])
fr_india.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(fr_india)

In [ ]:
fr_india = fr_india.drop(to_drop, axis=1)
to_rename = rename_columns(fr_india.columns)
fr_india.columns = to_rename
fr_india.head()

In [ ]:
cleaned_files.append(fr_india)

### FR_company_social_urls.csv

In [ ]:
files_list[5]

In [ ]:
fr_social_urls = pd.read_csv(files_list[5])
fr_social_urls.head()

In [ ]:
url_dataset = ["Facebook", "Twitter", "Instagram", "Pinterest"]
fr_social_urls['link'] = fr_social_urls[url_dataset].values.tolist()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(fr_social_urls)
to_drop.extend(url_dataset)

In [ ]:
fr_social_urls = fr_social_urls.drop(to_drop, axis=1)
to_rename = rename_columns(fr_social_urls.columns)
fr_social_urls.columns = to_rename
fr_social_urls.head()

In [ ]:
cleaned_files.append(fr_social_urls)

### FR_disfold.csv

In [ ]:
files_list[6]

In [ ]:
fr_disfold = pd.read_csv(files_list[6])
fr_disfold.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(fr_disfold)

In [ ]:
fr_disfold = fr_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(fr_disfold.columns)
fr_disfold.columns = to_rename

In [ ]:
fr_disfold.founded = fr_disfold.founded.str.extract(r'([\d]{4})')
fr_disfold.ceo = fr_disfold.ceo.apply(lambda x : str(x).replace('CEO: ', ""))
fr_disfold.employees = fr_disfold.employees.apply(lambda x : str(x).replace('Employees: ', '').replace(",", ""))
fr_disfold = fr_disfold.replace('nan', np.nan)

fr_disfold.founded = fr_disfold.founded.astype(object).astype('Int64')
fr_disfold.employees = fr_disfold.employees.astype(object).astype('Int64')
fr_disfold.head()

In [ ]:
cleaned_files.append(fr_disfold)

### FR_famcap_germany.csv

In [ ]:
files_list[7]

In [ ]:
fr_germany = pd.read_csv(files_list[7])
fr_germany.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(fr_germany)

In [ ]:
fr_germany = fr_germany.drop(to_drop, axis=1)
to_rename = rename_columns(fr_germany.columns)
fr_germany.columns = to_rename
fr_germany.head()

In [ ]:
fr_germany.revenue = fr_germany.revenue.apply(lambda x : float(str(x).replace(',', '')))
fr_germany.employees = fr_germany.employees.apply(lambda x : str(x).replace(',', ''))

fr_germany.revenue = fr_germany.revenue.astype(float).astype(int)
fr_germany.employees = fr_germany.employees.astype(object).astype('Int64')
fr_germany.founded = fr_germany.founded.astype(object).astype('Int64')
fr_germany.head()

In [ ]:
cleaned_files.append(fr_germany)

### FR_followers.csv

In [ ]:
files_list[8]

In [ ]:
fr_followers = pd.read_csv(files_list[8])
fr_followers.head()

#### Dataset da non usare?

### FR_forbes.csv

In [ ]:
files_list[9]

In [ ]:
fr_forbes = pd.read_csv(files_list[9])
fr_forbes.head()

In [ ]:
to_rename = rename_columns(fr_forbes.columns)
fr_forbes.columns = to_rename

In [ ]:
fr_forbes.founded = fr_forbes.founded.astype(str).replace(r'\.0$', '', regex=True)
fr_forbes.country = fr_forbes.country.str.split(',', n=1).str.get(-1)
fr_forbes.revenue = fr_forbes.revenue.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)

fr_forbes = fr_forbes.replace('nan', np.nan)

fr_forbes.founded = fr_forbes.founded.astype(object).astype('Int64')
fr_forbes.revenue = fr_forbes.revenue.astype(object).astype('Int64')
fr_forbes.head()

In [ ]:
cleaned_files.append(fr_forbes)

### FR_ft.csv

In [ ]:
files_list[10]

In [ ]:
fr_ft = pd.read_csv(files_list[10])
fr_ft.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(fr_ft)

In [ ]:
fr_ft = fr_ft.drop(to_drop, axis=1)
to_rename = rename_columns(fr_ft.columns)
fr_ft.columns = to_rename

In [ ]:
fr_ft.revenue = fr_ft.revenue.apply(lambda x : (str(x).replace(',', '')))
fr_ft.employees = fr_ft.employees.astype(str).replace(r'\.0$', '', regex=True)
fr_ft = fr_ft.replace('nan', np.nan)

fr_ft.revenue = fr_ft.revenue.astype(object).astype('Int64')
fr_ft.employees = fr_ft.employees.astype(object).astype('Int64')
fr_ft.founded = fr_ft.founded.astype(object).astype('Int64')

fr_ft.head()

In [ ]:
cleaned_files.append(fr_ft)

### FR_sole24ore.csv

In [ ]:
files_list[11]

In [ ]:
fr_sole24 = pd.read_csv(files_list[11])
fr_sole24['country'] = 'ITALY'
fr_sole24.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(fr_sole24)
to_drop.append("address")

In [ ]:
fr_sole24 = fr_sole24.drop(to_drop, axis=1)
to_rename = rename_columns(fr_sole24.columns)
fr_sole24.columns = to_rename

In [ ]:
fr_sole24.employees = fr_sole24.employees.apply(lambda x : (str(x).replace('.', '')))
fr_sole24.employees = fr_sole24.employees.astype(object).astype('Int64')
fr_sole24.head()

In [ ]:
cleaned_files.append(fr_sole24)

### FR_valuetoday.csv

In [ ]:
files_list[12]

In [ ]:
fr_value = pd.read_csv(files_list[12])
fr_value.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(fr_value)

In [ ]:
fr_value = fr_value.drop(to_drop, axis=1)
to_rename = rename_columns(fr_value.columns)
fr_value.columns = to_rename

In [ ]:
fr_value.founded = fr_value.founded.astype(str).replace(r'\.0$', '', regex=True)
fr_value.employees = fr_value.employees.apply(lambda x : (str(x).replace(',', '')))
fr_value = fr_value.replace('nan', np.nan)

fr_value.employees = fr_value.employees.astype(object).astype('Int64')
fr_value.founded = fr_value.founded.astype(object).astype('Int64')

fr_value.head()

In [ ]:
cleaned_files.append(fr_value)

## GioPonSpiz

### GioPonSpiz-companiesmarketcap.com.json

In [ ]:
files_list[13]

In [ ]:
gps_companies = pd.read_json(files_list[13])
gps_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gps_companies)

In [ ]:
gps_companies = gps_companies.drop(to_drop, axis=1)
to_rename = rename_columns(gps_companies.columns)
gps_companies.columns = to_rename

In [ ]:
gps_companies.marketcap = gps_companies.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
gps_companies.marketcap = gps_companies.marketcap.astype(object).astype('Int64')
gps_companies.head()

In [ ]:
cleaned_files.append(gps_companies)

### GioPonSpiz-disfold.com.json

In [ ]:
files_list[14]

In [ ]:
gps_disfold = pd.read_json(files_list[14])
gps_disfold.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gps_disfold)
to_drop.append("industry")

In [ ]:
gps_disfold = gps_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(gps_disfold.columns)
gps_disfold.columns = to_rename

In [ ]:
gps_disfold = gps_disfold.replace("", np.nan)
gps_disfold.marketcap = gps_disfold.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
gps_disfold.marketcap = gps_disfold.marketcap.astype(object).astype('Int64')
gps_disfold.head()

In [ ]:
cleaned_files.append(gps_disfold)

### GioPonSpiz-value.today.json

In [ ]:
files_list[15]

In [ ]:
gps_value = pd.read_json(files_list[15])
gps_value.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gps_value)

In [ ]:
gps_value = gps_value.drop(to_drop, axis=1)
to_rename = rename_columns(gps_value.columns)
gps_value.columns = to_rename

In [ ]:
gps_value.marketcap = gps_value.marketcap.apply(lambda x: convert_market_cap_revenue(x.replace(',', '')) if pd.notnull(x) else x)
gps_value.marketcap = gps_value.marketcap.astype(object).astype('Int64')
gps_value.head()

In [ ]:
cleaned_files.append(gps_value)

### GioPonSpiz-wikipedia.org.json

In [ ]:
files_list[16]

In [ ]:
gps_wiki = pd.read_json(files_list[16])
gps_wiki.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gps_wiki)
to_drop.append("Industry")

In [ ]:
gps_wiki = gps_wiki.drop(to_drop, axis=1)
to_rename = rename_columns(gps_wiki.columns)
gps_wiki.columns = to_rename

In [ ]:
gps_wiki = gps_wiki.replace('', np.nan)
gps_wiki = gps_wiki.replace('UNKNOWN', np.nan)
gps_wiki = gps_wiki.replace('?', np.nan)
gps_wiki = gps_wiki.replace('-', np.nan)
gps_wiki.founded = gps_wiki.founded.astype(object).astype('Int64')
gps_wiki.head()

In [ ]:
cleaned_files.append(gps_wiki)

## MalPatSaj

### MalPatSaj-companiesmarketcap.com.xls

In [ ]:
files_list[17]

In [ ]:
mps_companies = pd.read_excel(files_list[17])
mps_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(mps_companies)

In [ ]:
mps_companies = mps_companies.drop(to_drop, axis=1)
to_rename = rename_columns(mps_companies.columns)
mps_companies.columns = to_rename

In [ ]:
mps_companies.marketcap = mps_companies.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
mps_companies.marketcap = mps_companies.marketcap.astype(object).astype('Int64')
mps_companies.head()

In [ ]:
cleaned_files.append(mps_companies)

### MalPatSaj-disfold.com.xlsx

In [ ]:
files_list[18]

In [ ]:
mps_disfold = pd.read_excel(files_list[18])
mps_disfold.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(mps_disfold)
to_drop.append("Industry")

In [ ]:
mps_disfold = mps_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(mps_disfold.columns)
mps_disfold.columns = to_rename

In [ ]:
mps_disfold.marketcap = mps_disfold.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
mps_disfold.marketcap = mps_disfold.marketcap.astype(object).astype('Int64')
mps_disfold.head()

In [ ]:
cleaned_files.append(mps_disfold)

### MalPatSaj-forbes.com.xls

In [ ]:
files_list[19]

In [ ]:
mps_forbes = pd.read_excel(files_list[19])
mps_forbes.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(mps_forbes)
to_drop.append("Assets")

In [ ]:
mps_forbes = mps_forbes.drop(to_drop, axis=1)
to_rename = rename_columns(mps_forbes.columns)
mps_forbes.columns = to_rename

In [ ]:
mps_forbes.revenue = mps_forbes.revenue.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
mps_forbes.revenue = mps_forbes.revenue.astype(object).astype('Int64')
mps_forbes.head()

In [ ]:
cleaned_files.append(mps_forbes)

### MalPatSaj-wikipedia.org.xls

In [ ]:
files_list[20]

In [ ]:
mps_wiki = pd.read_excel(files_list[20])
mps_wiki.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(mps_wiki)
to_drop.append("Industry")

In [ ]:
mps_wiki = mps_wiki.drop(to_drop, axis=1)
to_rename = rename_columns(mps_wiki.columns)
mps_wiki.columns = to_rename
mps_wiki.founded = mps_wiki.founded.str.extract(r'([\d]{4})')
mps_wiki.founded = mps_wiki.founded.astype(object).astype('Int64')
mps_wiki.head()

In [ ]:
cleaned_files.append(mps_wiki)

## MarScoToc

### MarScoToc-ambitionbox.com.csv

In [ ]:
files_list[21]

In [ ]:
mst_ambition = pd.read_csv(files_list[21], encoding="utf-8", encoding_errors="ignore")
mst_ambition.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(mst_ambition)

In [ ]:
mst_ambition = mst_ambition.drop(to_drop, axis=1)
to_rename = rename_columns(mst_ambition.columns)
mst_ambition.columns = to_rename

In [ ]:
mst_ambition.country = mst_ambition.country.apply(lambda x: str(x).split(',')[-1])
mst_ambition.founded = mst_ambition.founded.str.extract(r'([\d]{4})')
mst_ambition = mst_ambition.replace('nan', np.nan)

mst_ambition.founded = mst_ambition.founded.astype(object).astype('Int64')

mst_ambition.head()

In [ ]:
cleaned_files.append(mst_ambition)

### MarScoToc-companiesmarketcap.com.csv

In [ ]:
files_list[22]

In [ ]:
mst_companies = pd.read_csv(files_list[22], encoding="utf-8", encoding_errors="ignore")
mst_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(mst_companies)

In [ ]:
mst_companies = mst_companies.drop(to_drop, axis=1)
to_rename = rename_columns(mst_companies.columns)
mst_companies.columns = to_rename

In [ ]:
mst_companies.marketcap = mst_companies.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
mst_companies.marketcap = mst_companies.marketcap.astype(object).astype('Int64')
mst_companies.head()

In [ ]:
cleaned_files.append(mst_companies)

### MarScoToc-wikipedia.org.csv

In [ ]:
files_list[24]

In [ ]:
mst_wiki = pd.read_csv(files_list[24], encoding="utf-8", encoding_errors="ignore")
mst_wiki.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(mst_wiki)

In [ ]:
mst_wiki = mst_wiki.drop(to_drop, axis=1)
to_rename = rename_columns(mst_wiki.columns)
mst_wiki.columns = to_rename

In [ ]:
mst_wiki.country = mst_wiki.country.apply(lambda x: str(x).split(',')[-1])
mst_wiki.founded = mst_wiki.founded.str.extract(r'([\d]{4})')
mst_wiki = mst_wiki.replace('nan', np.nan)

mst_wiki.founded = mst_wiki.founded.astype(object).astype('Int64')

mst_wiki.head()

In [ ]:
cleaned_files.append(mst_wiki)

## avengers

### avengers-companiesmarketcap.jsonl

In [ ]:
files_list[25]

In [ ]:
avengers_companies = pd.read_json(files_list[25], lines=True)
avengers_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(avengers_companies)

In [ ]:
avengers_companies = avengers_companies.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_companies.columns)
avengers_companies.columns = to_rename

In [ ]:
avengers_companies.marketcap = avengers_companies.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
avengers_companies.marketcap = avengers_companies.marketcap.astype(object).astype('Int64')
avengers_companies.head()

In [ ]:
cleaned_files.append(avengers_companies)

### avengers-disfold.jsonl

In [ ]:
files_list[26]

In [ ]:
avengers_disfold = pd.read_json(files_list[26], lines=True)
avengers_disfold.head()

In [ ]:
avengers_disfold.gbp = avengers_disfold.gbp.fillna(avengers_disfold.market_cap)

In [ ]:
to_drop, _ = columns_to_remove_not_remove(avengers_disfold)
to_drop.append("market_cap")

In [ ]:
avengers_disfold = avengers_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_disfold.columns)
avengers_disfold.columns = to_rename

In [ ]:
avengers_disfold.employees = avengers_disfold.employees.apply(lambda x: str(x).replace(',', ''))
avengers_disfold.founded = avengers_disfold.founded.str.extract(r'([\d]{4})')
avengers_disfold.marketcap = avengers_disfold.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)

avengers_disfold = avengers_disfold.replace('nan', np.nan)

avengers_disfold.marketcap = avengers_disfold.marketcap.astype(object).astype('Int64')
avengers_disfold.employees = avengers_disfold.employees.astype(object).astype('Int64')
avengers_disfold.founded = avengers_disfold.founded.astype(object).astype('Int64')
avengers_disfold.head()

In [ ]:
cleaned_files.append(avengers_disfold)

### avengers-hithorizons.jsonl

In [ ]:
files_list[27]

In [ ]:
avengers_hit = pd.read_json(files_list[27], lines=True)
avengers_hit.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(avengers_hit)

In [ ]:
avengers_hit = avengers_hit.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_hit.columns)
avengers_hit.columns = to_rename

In [ ]:
avengers_hit.country = avengers_hit.country.apply(lambda x: str(x).split(',', 5)[-1])
avengers_hit.founded = avengers_hit.founded.astype(object).astype('Int64')

avengers_hit.head()

In [ ]:
cleaned_files.append(avengers_hit)

### avengers-valuetoday.jsonl

In [ ]:
files_list[28]

In [ ]:
avengers_value = pd.read_json(files_list[28], lines=True)
avengers_value.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(avengers_value)

In [ ]:
avengers_value = avengers_value.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_value.columns)
avengers_value.columns = to_rename

In [ ]:
avengers_value.employees = avengers_value.employees.apply(lambda x: str(x).replace(',', ''))
avengers_value.revenue = avengers_value.revenue.apply(lambda x: convert_market_cap_revenue(x.replace(',', '.')) if pd.notnull(x) else x)

avengers_value = avengers_value.replace('nan', np.nan)

avengers_value.employees = avengers_value.employees.astype(object).astype('Int64')
avengers_value.revenue = avengers_value.revenue.astype(object).astype('Int64')
avengers_value.head()

In [ ]:
cleaned_files.append(avengers_value)

## gren

### gren-companiesmarketcap.com.json

In [ ]:
files_list[29]

In [ ]:
gren_companies = pd.read_json(files_list[29])
gren_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gren_companies)

In [ ]:
gren_companies = gren_companies.drop(to_drop, axis=1)
to_rename = rename_columns(gren_companies.columns)
gren_companies.columns = to_rename
gren_companies = gren_companies.replace('', np.nan)
gren_companies.marketcap = gren_companies.marketcap.astype(object).astype('Int64')
gren_companies.head()

In [ ]:
cleaned_files.append(gren_companies)

### gren-disfold.com.json

In [ ]:
files_list[30]

In [ ]:
gren_disfold = pd.read_json(files_list[30])
gren_disfold.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gren_disfold)
to_drop.append("industry")

In [ ]:
gren_disfold = gren_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(gren_disfold.columns)
gren_disfold.columns = to_rename
gren_disfold.head()

In [ ]:
cleaned_files.append(gren_disfold)

### gren-ft.com.json

In [ ]:
files_list[31]

In [ ]:
gren_ft = pd.read_json(files_list[31])
gren_ft.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gren_ft)

In [ ]:
gren_ft = gren_ft.drop(to_drop, axis=1)
to_rename = rename_columns(gren_ft.columns)
gren_ft.columns = to_rename

In [ ]:
gren_ft = gren_ft.replace('n/a', np.nan)
gren_ft.employees = gren_ft.employees.astype(object).astype('Int64')
gren_ft.head()

In [ ]:
cleaned_files.append(gren_ft)

### gren-value.today.json

In [ ]:
files_list[32]

In [ ]:
gren_value = pd.read_json(files_list[32])
gren_value.head()

In [ ]:
url_dataset = ['wikipedia_page_url', 'twitter_page_url', 'facebook_page_url']
gren_value['link'] = gren_value[url_dataset].values.tolist()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(gren_value)
to_drop.extend(url_dataset)

In [ ]:
gren_value = gren_value.drop(to_drop, axis=1)
to_rename = rename_columns(gren_value.columns)
gren_value.columns = to_rename

In [ ]:
gren_value = gren_value.replace('', np.nan)
gren_value.revenue = gren_value.revenue.astype(object).astype('Int64')
gren_value.marketcap = gren_value.marketcap.astype(object).astype('Int64')
gren_value.employees = gren_value.employees.astype(object).astype('Int64')
gren_value.head()

In [ ]:
cleaned_files.append(gren_value)

## iGMM

### iGMM-cbinsights.com.json

In [ ]:
files_list[33]

In [ ]:
igmm_cbinsights = pd.read_json(files_list[33])
igmm_cbinsights.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(igmm_cbinsights)

In [ ]:
igmm_cbinsights = igmm_cbinsights.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_cbinsights.columns)
igmm_cbinsights.columns = to_rename
igmm_cbinsights.head()

In [ ]:
cleaned_files.append(igmm_cbinsights)

### iGMM-companiesmarketcap.com.json

In [ ]:
files_list[34]

In [ ]:
igmm_companies = pd.read_json(files_list[34])
igmm_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(igmm_companies)

In [ ]:
igmm_companies = igmm_companies.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_companies.columns)
igmm_companies.columns = to_rename

In [ ]:
igmm_companies.marketcap = igmm_companies.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
igmm_companies = igmm_companies.replace('N/A', np.nan) 
igmm_companies.marketcap = igmm_companies.marketcap.astype(object).astype('Int64')
igmm_companies.head()

In [ ]:
cleaned_files.append(igmm_companies)

### iGMM-disfold.com.json

In [ ]:
files_list[35]

In [ ]:
igmm_disfold = pd.read_json(files_list[35])
igmm_disfold.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(igmm_disfold)
to_drop.append("headquarters")
to_drop.append("revenue")
to_drop.append("industry")

In [ ]:
igmm_disfold = igmm_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_disfold.columns)
igmm_disfold.columns = to_rename

In [ ]:
igmm_disfold.employees = igmm_disfold.employees.apply(lambda x: str(x).replace(',', ''))
igmm_disfold.founded = igmm_disfold.founded.str.extract(r'([\d]{4})')
igmm_disfold.marketcap = igmm_disfold.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
igmm_disfold = igmm_disfold.replace('None', np.nan)

igmm_disfold.employees = igmm_disfold.employees.astype(object).astype('Int64')
igmm_disfold.founded = igmm_disfold.founded.astype(object).astype('Int64')
igmm_disfold.marketcap = igmm_disfold.marketcap.astype(object).astype('Int64')

igmm_disfold.head()

In [ ]:
cleaned_files.append(igmm_disfold)

### iGMM-value.today.json

In [ ]:
files_list[36]

In [ ]:
igmm_value = pd.read_json(files_list[36])
igmm_value.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(igmm_value)

In [ ]:
igmm_value = igmm_value.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_value.columns)
igmm_value.columns = to_rename

In [ ]:
igmm_value.employees = igmm_disfold.employees.apply(lambda x: str(x).replace(',', ''))
igmm_value.revenue = igmm_value.revenue.apply(lambda x: convert_market_cap_revenue(x.replace(',', '.')) if pd.notnull(x) else x)
igmm_value = igmm_value.replace('<NA>', np.nan)

igmm_value.employees = igmm_value.employees.astype(object).astype('Int64')
igmm_value.revenue = igmm_value.revenue.astype(object).astype('Int64')

igmm_value.head()

In [ ]:
cleaned_files.append(igmm_value)

## silvestri

### silvestri-disfold.com.csv

In [ ]:
files_list[37]

In [ ]:
silvestri_disfold = pd.read_csv(files_list[37])
silvestri_disfold.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(silvestri_disfold)

In [ ]:
silvestri_disfold = silvestri_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_disfold.columns)
silvestri_disfold.columns = to_rename
silvestri_disfold.head()

In [ ]:
cleaned_files.append(silvestri_disfold)

### silvestri-forbes.com.csv

In [ ]:
files_list[38]

In [ ]:
silvestri_forbes = pd.read_csv(files_list[38])
silvestri_forbes.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(silvestri_forbes)

In [ ]:
silvestri_forbes = silvestri_forbes.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_forbes.columns)
silvestri_forbes.columns = to_rename

In [ ]:
silvestri_forbes.revenue = silvestri_forbes.revenue.apply(lambda x: convert_market_cap_revenue(x.replace(',', '.')) if pd.notnull(x) else x)
silvestri_forbes.revenue = silvestri_forbes.revenue.astype(object).astype('Int64')
silvestri_forbes.head()

In [ ]:
cleaned_files.append(silvestri_forbes)

### silvestri-ft.com.csv

In [ ]:
files_list[39]

In [ ]:
silvestri_ft = pd.read_csv(files_list[39])
silvestri_ft.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(silvestri_ft)

In [ ]:
silvestri_ft = silvestri_ft.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_ft.columns)
silvestri_ft.columns = to_rename

In [ ]:
silvestri_ft.revenue = silvestri_ft.revenue.apply(lambda x : (str(x).replace(',', '')))
silvestri_ft.employees = silvestri_ft.employees.astype(str).replace(r'\.0$', '', regex=True)
silvestri_ft = silvestri_ft.replace('nan', np.nan)

silvestri_ft.revenue = silvestri_ft.revenue.astype(object).astype('Int64')
silvestri_ft.employees = silvestri_ft.employees.astype(object).astype('Int64')
silvestri_ft.founded = silvestri_ft.founded.astype(object).astype('Int64')

silvestri_ft.head()

In [ ]:
cleaned_files.append(silvestri_ft)

### silvestri-value.today.csv

In [ ]:
files_list[40]

In [ ]:
silvestri_value = pd.read_csv(files_list[40])
silvestri_value.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(silvestri_value)

In [ ]:
silvestri_value = silvestri_value.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_value.columns)
silvestri_value.columns = to_rename

In [ ]:
silvestri_value.employees = silvestri_value.employees.apply(lambda x : (str(x).replace(',', '')))
silvestri_value.revenue = silvestri_value.revenue.apply(lambda x: convert_market_cap_revenue(x.replace(',', '.')) if pd.notnull(x) else x)

silvestri_value = silvestri_value.replace('<NA>', np.nan)
silvestri_value = silvestri_value.replace('nan', np.nan)

silvestri_value.employees = silvestri_value.employees.astype(object).astype('Int64')
silvestri_value.revenue = silvestri_value.revenue.astype(object).astype('Int64')
silvestri_value.head()

In [ ]:
cleaned_files.append(silvestri_value)

## slytherin

### slytherin-disfold.com.json

In [ ]:
files_list[41]

In [ ]:
slytherin_disfold = pd.read_json(files_list[41])
slytherin_disfold.head()

#### Marketcap diverse valute

In [ ]:
to_drop, _ = columns_to_remove_not_remove(slytherin_disfold)
to_drop.append("link")
to_drop.append("market_cap")

In [ ]:
slytherin_disfold = slytherin_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_disfold.columns)
slytherin_disfold.columns = to_rename

In [ ]:
slytherin_disfold.country = slytherin_disfold.country.apply(lambda x : (str(x).split(',')[-1].strip()))
slytherin_disfold.employees = slytherin_disfold.employees.apply(lambda x : (str(x).replace(',', '')))
slytherin_disfold = slytherin_disfold.replace('Not found', np.nan)

slytherin_disfold.employees = slytherin_disfold.employees.astype(object).astype('Int64')

slytherin_disfold.head()

In [ ]:
cleaned_files.append(slytherin_disfold)

### slytherin-forbes.com.json

In [ ]:
files_list[42]

In [ ]:
slytherin_forbes = pd.read_json(files_list[42])
slytherin_forbes.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(slytherin_forbes)
to_drop.append("link")

In [ ]:
slytherin_forbes = slytherin_forbes.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_forbes.columns)
slytherin_forbes.columns = to_rename

In [ ]:
slytherin_forbes.employees = slytherin_forbes.employees.apply(lambda x : (str(x).replace(',', '')))
slytherin_forbes.revenue = slytherin_forbes.revenue.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
slytherin_forbes = slytherin_forbes.replace('Not found', np.nan)

slytherin_forbes.employees = slytherin_forbes.employees.astype(object).astype('Int64')
slytherin_forbes.revenue = slytherin_forbes.revenue.astype(object).astype('Int64')
slytherin_forbes.founded = slytherin_forbes.founded.astype(object).astype('Int64')

slytherin_forbes.head()

In [ ]:
cleaned_files.append(slytherin_forbes)

### slytherin-ft.com.json

In [ ]:
files_list[43]

In [ ]:
slytherin_ft = pd.read_json(files_list[43])
slytherin_ft.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(slytherin_ft)

In [ ]:
slytherin_ft = slytherin_ft.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_ft.columns)
slytherin_ft.columns = to_rename

In [ ]:
slytherin_ft.employees = slytherin_ft.employees.apply(lambda x : (str(x).replace('.', '').replace(',', '')))
slytherin_ft.revenue = slytherin_ft.revenue.apply(lambda x : (float(str(x).replace(',', ''))))
slytherin_ft = slytherin_ft.replace('Not found', np.nan)
slytherin_ft = slytherin_ft.replace('nan', np.nan)

slytherin_ft.revenue = slytherin_ft.revenue.astype(float).astype(int)
slytherin_ft.employees = slytherin_ft.employees.astype(object).astype('Int64')
slytherin_ft.founded = slytherin_ft.founded.astype(object).astype('Int64')

slytherin_ft.head()

In [ ]:
cleaned_files.append(slytherin_ft)

### slytherin-value.today.json

In [ ]:
files_list[44]

In [ ]:
slytherin_value = pd.read_json(files_list[44])
slytherin_value.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(slytherin_value)
to_drop.append("link")

In [ ]:
slytherin_value = slytherin_value.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_value.columns)
slytherin_value.columns = to_rename

In [ ]:
slytherin_value.founded = slytherin_value.founded.str.extract(r'([\d]{4})')
slytherin_value.revenue = slytherin_value.revenue.str.extract('([\d]+[.,\d]+)', expand=False)
slytherin_value.revenue = slytherin_value.revenue.apply(lambda x: convert_market_cap_revenue(x.replace('.', '').replace(',', '.') + 'Million') if pd.notnull(x) else x)

slytherin_value = slytherin_value.replace('Not found', np.nan)
slytherin_value = slytherin_value.replace('nan', np.nan)

slytherin_value.founded = slytherin_value.founded.astype(object).astype('Int64')
slytherin_value.revenue = slytherin_value.revenue.astype(object).astype('Int64')
slytherin_value.head()

In [ ]:
cleaned_files.append(slytherin_value)

## wissel

### wissel-ariregister.csv

In [ ]:
files_list[45]

In [ ]:
wissel_ariregister = pd.read_csv(files_list[45])
wissel_ariregister.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(wissel_ariregister)

In [ ]:
wissel_ariregister = wissel_ariregister.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_ariregister.columns)
wissel_ariregister.columns = to_rename
wissel_ariregister.head()

In [ ]:
cleaned_files.append(wissel_ariregister)

### wissel-companiesmarketcap.com.csv

In [ ]:
files_list[46]

In [ ]:
wissel_companies = pd.read_csv(files_list[46])
wissel_companies.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(wissel_companies)

In [ ]:
wissel_companies = wissel_companies.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_companies.columns)
wissel_companies.columns = to_rename

In [ ]:
wissel_companies.employees = wissel_companies.employees.apply(lambda x : (str(x).replace('.', '')))
wissel_companies.revenue = wissel_companies.revenue.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)
wissel_companies.marketcap = wissel_companies.marketcap.apply(lambda x: convert_market_cap_revenue(x) if pd.notnull(x) else x)

wissel_companies = wissel_companies.replace('<NA>', np.nan)
wissel_companies = wissel_companies.replace('nan', np.nan)

wissel_companies.employees = wissel_companies.employees.astype(object).astype('Int64')
wissel_companies.revenue = wissel_companies.revenue.astype(object).astype('Int64')
wissel_companies.marketcap = wissel_companies.marketcap.astype(object).astype('Int64')

wissel_companies.head()

In [ ]:
cleaned_files.append(wissel_companies)

### wissel-gov.uk.csv

In [ ]:
files_list[47]

In [ ]:
wissel_gov = pd.read_csv(files_list[47])
wissel_gov.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(wissel_gov)

In [ ]:
wissel_gov = wissel_gov.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_gov.columns)
wissel_gov.columns = to_rename
wissel_gov.head()

In [ ]:
cleaned_files.append(wissel_gov)

### wissel-info-clipper.com.csv

In [ ]:
files_list[48]

In [ ]:
wissel_info = pd.read_csv(files_list[48])
wissel_info.head()

In [ ]:
to_drop, _ = columns_to_remove_not_remove(wissel_info)

In [ ]:
wissel_info = wissel_info.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_info.columns)
wissel_info.columns = to_rename
wissel_info.head()

In [ ]:
cleaned_files.append(wissel_info)

# Datasets Concatenation

In [ ]:
final = pd.concat(cleaned_files, ignore_index=True)

In [ ]:
for c in final.columns:
    final[str(c)] = final[str(c)].replace(r'\r+|\n+|\t+',' ', regex=True)

    if c == 'name':
        final[str(c)] = final[str(c)].str.replace('[^a-zA-Z0-9]', ' ', regex=True)
        final[str(c)] = final[str(c)].str.replace(r'\s+', ' ', regex=True)

    if c == 'sector' or c == 'country' or c == 'ceo':
        final[str(c)] = final[str(c)].str.replace('[^a-zA-Z]', ' ', regex=True)
        final[str(c)] = final[str(c)].str.replace(r'\s+', ' ', regex=True)
        final[str(c)] = final[str(c)].str.replace('nan', '')
    
    if final[str(c)].dtypes != 'Int64':
        final[str(c)] = final[str(c)].str.strip()
        final[str(c)] = final[str(c)].str.upper()
    
    final[str(c)] = final[str(c)].replace('', pd.NA)
    final[str(c)] = final[str(c)].fillna(pd.NA)

In [ ]:
final.to_csv("./src/1. aligned_dataset.csv")

In [ ]:
t_end = time.time()
print(t_end-t_start)

## Report final dataset

In [ ]:
from pandas_profiling import ProfileReport
profile=ProfileReport(final)
profile.to_notebook_iframe()